# Principaux modules d'OpenFisca

- OpenFisca-Core : moteur d'OpenFisca
- OpenFisca-France : législation socio-fiscale française
- OpenFisca-Web-API : API web (utilise OpenFisca-Core + OpenFisca-France)
- OpenFisca-Web-UI : Simulateur web d'OpenFisca (appelle OpenFisca-Web-API en HTTP)
- OpenFisca-France-Reform-Landais-Piketty-Saez : Un exemple de réforme

Les sources des modules : https://github.com/openfisca/

# Utilisation simplifiée d'OpenFisca

In [3]:
from datetime import date

from openfisca_france import init_country
from openfisca_france.model.base import *

ImportError: cannot import name init_country

## Adaptation pour faciliter l'usage de ce notebook

_Ce correctif permet de redéfinir plusieurs fois la même variable sans provoquer d'erreur._

In [3]:
import functools

from openfisca_core.formulas import make_reference_formula_decorator
from openfisca_france.entities import entity_class_by_symbol

reference_formula = make_reference_formula_decorator(entity_class_by_symbol = entity_class_by_symbol, update = True)
reference_input_variable = functools.partial(reference_input_variable, update = True)

NameError: name 'make_reference_formula_decorator' is not defined

# Variable d'entrée

In [29]:
reference_input_variable(
    column = FloatCol,  # le type de variable : BoolCol, FloatCol, EnumCol, IntCol, StrCol...
    entity_class = Individus,  # Individus, Familles, FoyersFiscaux ou Menages
    label = "Salaire brut",
    name = 'salaire_brut',
    )

# Variable avec formule

In [30]:
@reference_formula
class salaire_net(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Salaire net"

    def function(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        salaire_brut = simulation.calculate('salaire_brut', period)

        return period, salaire_brut * 0.8

In [31]:
@reference_formula
class salaire_imposable(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Salaire imposable"

    def function(self, simulation, period):
        period = period.start.period(u'year').offset('first-of')
        salaire_net = simulation.calculate('salaire_net', period)

        return period, salaire_net * 0.9

# Variable avec différentes formules en fonction de la date

In [32]:
@reference_formula
class revenu_social_activite(DatedFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"RSA"

    @dated_function(date(2010, 1, 1), date(2010, 12, 31))
    def function_2010(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        salaire_net = simulation.calculate('salaire_net', period)
        age = simulation.calculate('age', period)

        return period, (age >= 18) * (salaire_net < 500) * 100.0

    @dated_function(date(2011, 1, 1))
    def function_2011_2014(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        salaire_net = simulation.calculate('salaire_net', period)
        age = simulation.calculate('age', period)

        return period, (age >= 18) * (salaire_net < 600) * 200.0

# Système socio-fiscal

In [33]:
TaxBenefitSystem = init_country()
tax_benefit_system = TaxBenefitSystem()

# Simulation

In [34]:
simulation = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_brut = 12000,
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_brut = 6000,
        ),
    enfants = [
        dict(
            birth = date(2014, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [35]:
simulation.calculate('salaire_net')

array([ 9600.,  4800.,     0.], dtype=float32)

In [36]:
simulation.calculate('salaire_imposable')

array([ 8640.,  4320.,     0.], dtype=float32)

In [37]:
simulation.calculate('salaire_net', '2014-12')

array([ 800.,  400.,    0.], dtype=float32)

In [38]:
simulation.calculate('revenu_social_activite', '2014-12')

array([   0.,  200.,    0.], dtype=float32)

In [2]:
simulation.calculate('revenu_social_activite')

NameError: name 'simulation' is not defined

# Réforme

In [40]:
from openfisca_core import reforms

In [41]:
ReformeRevenuDeBase = reforms.make_reform(
    name = u"Réforme Revenu de base",
    reference = tax_benefit_system,
    )

In [47]:
ReformeRevenuDeBase.input_variable(
    column = FloatCol,
    entity_class = Individus,  # Individus, Familles, FoyersFiscaux ou Menages
    label = u"Chômage brut",
    name = 'chomage_brut',
    )

In [43]:
@ReformeRevenuDeBase.formula
class chomage_net(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Chômage net"

    def function(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        chomage_brut = simulation.calculate('chomage_brut', period)

        return period, chomage_brut * 0.8

In [44]:
@ReformeRevenuDeBase.formula
class revenu_net(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Revenu net"

    def function(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        chomage_net = simulation.calculate('chomage_net', period)
        salaire_net = simulation.calculate('salaire_net', period)

        return period, chomage_net + salaire_net

In [45]:
reform = ReformeRevenuDeBase()

In [46]:
reform_simulation = reform.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_brut = 12000,
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        chomage_brut = 6000,
        ),
    enfants = [
        dict(
            birth = date(2014, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

ValueError: {'test_case': {'individus': {'ind1': {'chomage_brut': u'Unexpected item'}}}} for: {'axes': None, 'period': Period((u'year', Instant((2014, 1, 1)), 1)), 'test_case': {'foyers_fiscaux': OrderedDict([(0, {'personnes_a_charge': ['ind2'], 'declarants': ['ind0', 'ind1']})]), 'individus': OrderedDict([('ind0', {u'salaire_brut': 12000.0, u'birth': datetime.date(1980, 1, 1)}), ('ind1', {u'birth': datetime.date(1980, 1, 1), 'chomage_brut': 6000}), ('ind2', {u'birth': datetime.date(2014, 1, 1)})]), 'familles': OrderedDict([(0, {'parents': ['ind0', 'ind1'], 'enfants': ['ind2']})]), 'menages': OrderedDict([(0, {'conjoint': 'ind1', 'autres': [], 'enfants': ['ind2'], 'personne_de_reference': 'ind0'})])}}

In [ ]:
reform_simulation.calculate('salaire_net', '2014-12')

In [ ]:
reform_simulation.calculate('chomage_net', '2014-12')

In [ ]:
reform_simulation.calculate('revenu_net', '2014-12')